In [1]:
# Implements（如何实现）
#     Read Data
#     Pre-processing & View data 画图不会，反正数据本身很糟糕
#     Get CF typr by ranking RMSE 和 MAE
#     Parameter Estimate: Grid Search(已用的)； 
#                       Naive Baysian 和 gradient descent(不会)；但是要是会了分肯定高（lab 有代码）
#     现在还需要把book的名字和id(ISBN)给连回去。。。实现一个比较美观？的UI
# Experiments:（重点得分）
#     比较各个factor（u_id, b_id）对于rating的影响，哪个比较大
#     比较前面设置的参数是否有影响（min_book_ratings； data clean（虽然我并不懂怎么clean））
#     等等
# 关于surprise的问题自己看： https://surprise.readthedocs.io/en/stable/getting_started.html 
# 源代码： https://github.com/NicolasHug/Surprise/tree/711fb80748140c44e0ed870e573c735307e6c3cc

In [2]:
##可以参照5826老师把图画好看点。。。

In [3]:
import pandas as pd, numpy as np, os, gc,\
matplotlib.pyplot as plt,seaborn as sb,re,warnings, calendar, sys
from numpy import arange
get_ipython().run_line_magic('matplotlib', 'inline')
warnings.filterwarnings('ignore'); 

#reference:https://bmanohar16.github.io/blog/recsys-evaluation-in-surprise


global directory; directory = '../BookRating'
def files(): return os.listdir(directory)

#reading data
user = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'Ratings']
df = pd.merge(user, ratings, on='userID', how='inner')
df.drop(['Location', 'Age'], axis=1, inplace=True)

df.head(10)
#pandas 自带了describe和summary。。。不需要用5826了
df.describe()

,userID,Ratings
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [4]:
#preprocessing data
#这个min_book_ratings是否对后面有影响有待查验，你们可以换个值跑一下
#这里需要画图
min_book_ratings = 50
filter_books = df['ISBN'].value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

min_user_ratings = 50
filter_users = df['userID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['ISBN'].isin(filter_books)) & (df['userID'].isin(filter_users))]
#df_new = df_new.fillna(0)
df_new.head(10)

,userID,ISBN,Ratings
394,243,0060915544,10
395,243,0060977493,7
397,243,0156006529,0
400,243,0316096199,0
401,243,0316601950,9
402,243,0316602906,0
403,243,0316666343,0
404,243,0316693006,0
405,243,0316776963,9
406,243,0316899984,7


In [5]:
print('No. of Unique Users    :', df_new.userID.nunique())
print('No. of Unique ISBN :', df_new.ISBN.nunique())
print('Unique Ratings  :', df_new.Ratings.unique())
#since the unique value of ratings is 11: 0 - 10, no NaN
ratings_per_user = df_new['Ratings'].count()/df_new.userID.nunique()
print('ratings per user',ratings_per_user)
ratings_per_book = df_new['Ratings'].count()/df_new.ISBN.nunique()
print('ratings per book',ratings_per_book)

No. of Unique Users    : 3237
No. of Unique ISBN : 2125
Unique Ratings  : [10  7  0  9  6  8  5  4  1  2  3]
ratings per user 43.4093296261971
ratings per book 66.12517647058823


In [6]:
ratings_matrix = df_new.pivot(index='userID', columns='ISBN', values='Ratings')
userID = ratings_matrix.index
bookID = ratings_matrix.columns

In [8]:
from surprise import Dataset
from surprise import Reader
#Smiliarity module
from surprise import similarities
#Prediction algorithm
#Basic algorithms
from surprise import AlgoBase
#k-NN inspired algorithms
#A basic collaborative filtering algorithm.
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import NMF
#matrix_factorization based
from surprise import SVD
from surprise import SVDpp
from surprise import CoClustering
#accuracy: rmse, mae
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

In [9]:
#Surprise Read
#scale 是0 - 10，上面我写了，那个网站写错了吧
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_new[['userID','ISBN','Ratings']], reader)
#cv = 2！！！！不是3！！！因为rating不是一个classifier！！！
#accuracy test for collaborative filtering algorithm based on Machine Learning
#reference to susanli2016/Machine-Learning-with-Python/Building Recommender System with Surprise
def test_algo(algorithm_list):
    benchmark = []
    for algorithm in algorithm_list:
        # Cross validation
        results = cross_validate(algorithm, data, cv=2, verbose=False)

        # Get results & append algorithm name
        rmse = pd.DataFrame(results, columns = ['test_rmse', 'test_mae']).mean(axis=0)
        rmse = rmse.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], \
                                     index=['Algorithm']))
        benchmark.append(rmse)
    surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
    return surprise_results


In [10]:
#这边最好再跑一下。。。
algorithm_list = [SVD(), SVDpp(), NMF(), KNNBasic(), \
                  KNNWithMeans(), KNNWithZScore(), CoClustering()]
test_algo(algorithm_list)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,test_rmse,test_mae
Algorithm,,
SVD,3.519448,2.758088
CoClustering,3.533304,2.646740
KNNWithMeans,3.556443,2.679320
KNNWithZScore,3.587723,2.658476
SVDpp,3.787355,2.811349
KNNBasic,3.823998,2.910460
NMF,3.973226,2.801949


In [11]:
#Co-clustering
#Param_grid
#试图用naive bayes找但是失败了==。
# from sklearn.model_selection import train_test_split as t_t_s
#上面这个module的名字和surprise库的重合了==。uncomment以后会跑不出来
from sklearn.naive_bayes import GaussianNB

#X_train, X_test, y_train, y_test = t_t_s(userID, bookID, test_size=0.25,random_state=0)
# fit a GaussianNB model

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

NameError: name 'X_train' is not defined

In [12]:
# 将数据随机分为训练和测试数据集
trainset, testset = train_test_split(data, test_size=.25)

In [13]:
#'40：ratings per user; 60: ratings per book(item)
#这个para_grid 不知道怎么找
#我没跑完==。跑不出来 没报错电脑太渣了
param_grid = {'n_cltr_u': [5,10,15,20,25,30,35,40,45,50], 'n_cltr_i': [5,10,15,20,25,30,35,40,45,50],
              'n_epochs': [20,25]}
CoCluster_gs = GridSearchCV(CoClustering, param_grid, measures=['rmse', 'mae'], cv=2)

CoCluster_gs.fit(data)


print(CoCluster_gs.best_score['rmse'])
print(CoCluster_gs.best_params['rmse'])

# algo = CoCluster_gs.best_estimator['rmse']
# predictions = algo.fit(trainset).test(testset)
# accuracy.mae(predictions)
# accuracy.rmse(predictions)

3.576759551898977
{'n_cltr_u': 5, 'n_cltr_i': 5, 'n_epochs': 20}


In [352]:
#KNN Means
##https://surprise.readthedocs.io/en/stable/getting_started.html#tuning-algorithm-parameters
param_grid = {'bsl_options': {'method': ['als', 'sgd'],
                              },
              'k': [20, 40, 60,80,100,200,300],
              'sim_options': {'name': ['cosine','pearson'],
                              'min_support': [10,50],
                              'user_based': [False]}
              }


knnmeans_gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=2)
knnmeans_gs.fit(data)

# best RMSE score
print(knnmeans_gs.best_score['rmse'])
print(knnmeans_gs.best_params['rmse'])

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computi

Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [14]:
#用训练集再跑一下
#the best_estimator attribute gives us an algorithm instance with the optimal set of parameters
algo = knnmeans_gs.best_estimator['rmse']
predictions = algo.fit(trainset).test(testset)
accuracy.mae(predictions)
accuracy.rmse(predictions)

NameError: name 'knnmeans_gs' is not defined